# Advent of Code 2024: Day 16
https://adventofcode.com/2024/day/16


## Part 1
Find the shortest path from the start to the end

In [1]:
def make_map(data: str):
    with open(data, "r") as f:
        data = f.read()
    data_map = []
    for line in data.split("\n"):
        data_map.append([c for c in line])
    return data_map


posWithDir = tuple[int, int, str]
data_map = make_map("input.txt")
DIRS = {
    "right": (0, 1),
    "left": (0, -1),
    "down": (1, 0),
    "up": (-1, 0),
}
TURNING = {
    "up": ["left", "right"],
    "right": ["up", "down"],
    "down": ["left", "right"],
    "left": ["up", "down"],
}
MAX_Y = len(data_map)
MAX_X = len(data_map[0])


def get_pos_by_value(data: list[list[str]], value: str) -> list[tuple[int, int] | None]:
    pos = []
    for i in range(MAX_Y):
        for j in range(MAX_X):
            if data[i][j] == value:
                pos.append((i, j))
    return pos


def _update_position(pos: posWithDir) -> posWithDir:
    return tuple(y + x for y, x in zip(pos[:2], DIRS[pos[2]])) + (pos[2],)


def _get_valid_neighbors(data: list[list[str]], pos: posWithDir) -> list[posWithDir]:
    valid_neighbors = []
    new_pos = _update_position(pos)
    if data[new_pos[0]][new_pos[1]] != "#":
        valid_neighbors.append(new_pos)
    for turn in TURNING[pos[2]]:
        valid_neighbors.append((pos[0], pos[1], turn))
    return valid_neighbors


def bfs(data: list[list[str]], start: posWithDir, end: tuple[int, int]):
    explore_q = [start]
    visited = set()
    visited.add(start)
    prev = {start: start}
    dist = {start: 0}
    while explore_q:
        cur = explore_q.pop()
        neighbors = _get_valid_neighbors(data, cur)
        for neighbor in neighbors:
            if neighbor[:2] == cur[:2]:
                new_dist = dist[cur] + 1000
            else:
                new_dist = dist[cur] + 1
            if neighbor not in visited:
                prev[neighbor] = cur
                dist[neighbor] = new_dist
                explore_q.append(neighbor)
                visited.add(neighbor)
            else:
                if new_dist < dist[neighbor]:
                    prev[neighbor] = cur
                    dist[neighbor] = new_dist
                    explore_q.append(neighbor)
    min_dist = None
    for p, d in dist.items():
        if p[:2] == end:
            if min_dist:
                min_dist = min(min_dist, d)
            else:
                min_dist = d
    return min_dist, dist


start = get_pos_by_value(data_map, "S")[0] + ("right",)
end = get_pos_by_value(data_map, "E")[0]

min_dist, dist = bfs(data_map, start, end)
min_dist

104516

## Part 2
Find the how many paths are part of at least one of the shortest paths

In [2]:
DIRS_REV = {
    "right": (0, -1),
    "left": (0, 1),
    "down": (-1, 0),
    "up": (1, 0),
}


def _get_valid_neighbors(
    data: dict[posWithDir, int], pos: posWithDir, dist: int
) -> list[posWithDir]:
    valid_neighbors = []
    for dir, dir_tuple in DIRS_REV.items():
        new_pos = tuple(y + x for y, x in zip(pos[:2], dir_tuple)) + (dir,)
        if data.get(new_pos) is not None:
            if data[new_pos] == dist - 1:
                valid_neighbors.append(new_pos)
    for dir in DIRS_REV:
        new_pos = pos[:2] + (dir,)
        if data[new_pos] == dist - 1000:
            valid_neighbors.append(new_pos)
    return valid_neighbors


pos = set()
pos_unique = set()
ends = [p for p, d in dist.items() if d == min_dist and p[:2] == end]
pos.add(ends[0])
pos_unique.add(ends[0][:2])
while ends:
    cur = ends.pop()
    neighbors = _get_valid_neighbors(dist, cur, dist[cur])
    for neighbor in neighbors:
        if neighbor not in pos:
            pos.add(neighbor)
            pos_unique.add(neighbor[:2])
            ends.append(neighbor)
print(len(pos_unique))

545
